In [10]:
file = open('./data/FullData/fullData.dat','r')

In [11]:
#for line in file:
#    print("{}".format(line.strip()))

In [15]:
file = open('./data/FullData/fullData.dat','r')
line = file.readline()
print("{}".format(line.strip('co')))
    

# co2a0000364.rd



In [25]:
tmp = line.split('2')
print(tmp[1][0])

a


In [ ]:
# Opening file
file1 = open('myfile.txt', 'r')
count = 0
 
# Using for loop
print("Using for loop")
for line in file1:
    count += 1
    print("Line{}: {}".format(count, line.strip()))
 
# Closing files
file1.close()